<a href="https://colab.research.google.com/github/tijlk/tmdb_box_office/blob/master/TMDB_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TMDB Box Office Prediction

We are challenging you to beat the other teams in two competitions. One of them is this one where you are asked to predict the box office results. For more info, see the [competition page](https://www.kaggle.com/c/tmdb-box-office-prediction).

## Setting up Google Colab

*   First of all, it's probably a good idea to save this notebook in your Google Drive. To do that, go to File and click on 'Save a copy in Drive'. Otherwise, you might lose your results if you're not careful.
*   Secondly, if you want to use GPU's, make sure to select a GPU runtime. Go to 'Runtime' -> 'Change runtime type'. Select 'GPU' as Hardware accelarator and click on 'Save'.

## Important! Configure your team name.
This will be used to identify your submissions in the Kaggle contests.

In [0]:
team_name = 'Avengers'

## Setting up the Kaggle API

All the teams are using the same Kaggle account, so that we can easily keep track of each other's scores. And so you don't have to set up anything :).

In [0]:
import os
os.environ["KAGGLE_USERNAME"] = "uniteds"
os.environ["KAGGLE_KEY"] = "e2cc23b4870d3b069e2f8bf9d159847d"

## Downloading the data

In [0]:
# Download the dataset.
!kaggle competitions download -c tmdb-box-office-prediction
!unzip -o train.csv.zip
!unzip -o test.csv.zip
!chmod 644 *.csv

## Loading Data

In [0]:
import numpy as np
import pandas as pd
from sklearn.model_selection import cross_validate
from sklearn.linear_model import LinearRegression
import os

In [0]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

print('train dataset size:', train.shape)
print('test dataset size:', test.shape)

train.sample(4)

## Feature engineering

In [0]:
train.info()

### Process JSON-style features

There are 8 JSON-style features, 4 numerical, 4 text, and 1 date feature. At first, convert JSON-styled features into string/category/list ones.

* **`belongs_to_collection`**: convert `name` into string
* **`genres`, `production_companies`**: convert `name` values into comma-separated string list
* **`production_countries`**: convert `iso_3166_1` values into comma-separated string list
* **`spoken_languages`**: convert `iso_639_1` values into comma-separated string list
* **`Keywords`**: convert `name` values into comma-separated string list
* **`cast`, `crew`**: get their lengths, as its detailed information is very unlikely relevant to the revenue 

In [0]:
def proc_json(string, key):
    try:
        data = eval(string)
        return ",".join([d[key] for d in data])
    except:
        return ''

def proc_json_len(string):
    try:
        data = eval(string)
        return len(data)
    except:
        return 0

train.belongs_to_collection = train.belongs_to_collection.apply(lambda x: proc_json(x, 'name'))
test.belongs_to_collection = test.belongs_to_collection.apply(lambda x: proc_json(x, 'name'))

train.genres = train.genres.apply(lambda x: proc_json(x, 'name'))
test.genres = test.genres.apply(lambda x: proc_json(x, 'name'))

train.production_companies = train.production_companies.apply(lambda x: proc_json(x, 'name'))
test.production_companies = test.production_companies.apply(lambda x: proc_json(x, 'name'))

train.production_countries = train.production_countries.apply(lambda x: proc_json(x, 'iso_3166_1'))
test.production_countries = test.production_countries.apply(lambda x: proc_json(x, 'iso_3166_1'))

train.spoken_languages = train.spoken_languages.apply(lambda x: proc_json(x, 'iso_639_1'))
test.spoken_languages = test.spoken_languages.apply(lambda x: proc_json(x, 'iso_639_1'))

train.Keywords = train.Keywords.apply(lambda x: proc_json(x, 'name'))
test.Keywords = test.Keywords.apply(lambda x: proc_json(x, 'name'))

train.cast = train.cast.apply(proc_json_len)
test.cast = test.cast.apply(proc_json_len)

train.crew = train.crew.apply(proc_json_len)
test.crew = test.crew.apply(proc_json_len)

### Genres
As a movie can have multiple genres, it is not a reasonable way to convert `genres` column into category type. It might make the same genres different, e.g. 'Drama,Romance' and 'Romance,Drama' would be categorized differently. Therefore we make dummy columns for all of the genres.

In [0]:
# get total genres list
genres = []
for idx, val in train.genres.iteritems():
    gen_list = val.split(',')
    for gen in gen_list:
        if gen == '':
            continue

        if gen not in genres:
            genres.append(gen)
            

genre_column_names = []
for gen in genres:
    col_name = 'genre_' + gen.replace(' ', '_')
    train[col_name] = train.genres.str.contains(gen).astype('uint8')
    test[col_name] = test.genres.str.contains(gen).astype('uint8')
    genre_column_names.append(col_name)

### Normalize Revenue and Budget

Budget and revenue are highly skewed and they need to be normalized by logarithm.

In [0]:
train['revenue'] = np.log1p(train['revenue'])
train['budget'] = np.log1p(train['budget'])
test['budget'] = np.log1p(test['budget'])

train.sample(5)

## Select features

For this benchmark solution, we are manually selecting the following features:

In [0]:
features = ['budget', 'popularity', 'runtime', 'genre_Adventure', 'genre_Action', 'genre_Fantasy', 
            'genre_Drama', 'genre_Family', 'genre_Animation', 'genre_Science_Fiction']

## Fix missing values

The `runtime` column has 2 and 4 missing values for the train and test dataset respectively:

In [0]:
print('-'*30)
print(train[features].isnull().sum())
print('-'*30)
print(test[features].isnull().sum())

So let's fill the missing values with the mean of the other runtimes.

In [0]:
train.runtime = train.runtime.fillna(np.mean(train.runtime))
test.runtime = test.runtime.fillna(np.mean(train.runtime))

In [0]:
train[features].sample(5)

## Train model

Let's use cross validation on a linear regression model using the selected features to check the explained variance:

In [0]:
X, y = train[features], train['revenue']
model = LinearRegression()
result = cross_validate(model, X, y, cv=10, scoring="explained_variance", verbose=False, n_jobs=-1)
print(f"The variance explained is {np.mean(result['test_score']):.1%}")

## Make predictions

Now let's train the model and make predictions for the test set:

In [0]:
model.fit(X, y)
predict = model.predict(test[features])

## Create submission file

Create a submission file from the predictions:

In [0]:
submit = pd.DataFrame({'id': test.id, 'revenue':np.expm1(predict)})
submit.to_csv('submission.csv', index=False)

----

## Submit your results to Kaggle

**IMPORTANT** Each team is allowed only **2** submissions per day per competition! So, be careful!

Run the following command:

In [0]:
if input("Are you sure you want to submit? (y)es or (n)o: ") == 'y':
    !kaggle competitions submit tmdb-box-office-prediction -f submission.csv -m "{team_name}"

In [0]:
!kaggle competitions submissions tmdb-box-office-prediction

So, did you beat the other teams? :)